# 필요한 모듈 import

In [1]:
from pynq import Overlay
import pickle, struct, time, math
import numpy as np
import matplotlib.pyplot as plt

# 데이터 불러오기
```pickle```모듈을 이용하여 변수 ```test_images```, ```test_images```, ```weightList```로 불러옵니다.

In [2]:
with open("weight", "rb") as file:
    weightList = pickle.load(file)

with open("test_images", "rb") as file:
    test_images = pickle.load(file)

with open("test_labels", "rb") as file:
    test_labels = pickle.load(file)

UnicodeDecodeError: 'ascii' codec can't decode byte 0x8a in position 0: ordinal not in range(128)

# 주소 dictionary
x[프로젝트 이름]_hw.h 파일에서 주소를 확인해서 dictionary로 정리합니다.

In [ ]:
poolAddress = dict()
poolAddress["ifMap"] = 0x8000
poolAddress["ofMap"] = 0x10000

convAddress = dict()
convAddress["ifMap"] = 0x2000
convAddress["ofMap"] = 0x8000
convAddress["_filter"] = 0x40000
convAddress["bias"] = 0x80000

fullAddress = dict()
fullAddress["ifMap"] = 0x200
fullAddress["ofMap"] = 0x400
fullAddress["weight"] = 0x10000
fullAddress["bias"] = 0x20000

# Convolution Neural Network 클래스
생성자에서 bitstream을 읽고 정상적으로 IP core가 호출되도록 최대 20초까지 기다립니다.

In [ ]:
class CNN:
    def __init__(self):
        self.overlay = None
        self.overlay = Overlay("/home/xilinx/pynq/overlays/cnn/cnn.bit")
        t2 = t1 = time.time()       
        while 1:
            t2 = time.time()
            if t2 - t1 > 20:
                print("Time out(20s).")
                break
            try:
                self.conv_ip = self.overlay.conv_0
                self.full_ip = self.overlay.full_0
                self.pool_ip = self.overlay.pool_0
                break
            except:
                continue
    
    def pool(self, ifMap, U, C, H, R):
        pass

    def conv(self, ifMap, _filter, bias, U, C, H, R, M):
        pass

    def full(self, ifMap, weight, bias, C, M):
        pass

    def poolWrite(self, ifMap, C, H, U, R):
        pass

    def poolRead(self, C, E):
        pass

    def convWrite(self, ifMap, _filter, bias, H, M, C, R, U):
        pass

    def convRead(self, M, E):
        pass

output featuer map 크기인 ```E```를 구하고 IP core에 write하고 read한 값을 반환합니다.

In [ ]:
def pool(self, ifMap, U, C, H, R):
    E = int((H - R) / U) + 1 
    self.poolWrite(ifMap, C, H, U, R)
    return self.poolRead(C, E)

def conv(self, ifMap, _filter, bias, U, C, H, R, M):
    E = int((H - R) / U) + 1
    self.convWrite(ifMap, _filter, bias, H, M, C, R, U)
    return self.convRead(M, E)

def full(self, ifMap, weight, bias, C, M):
    self.fullWrite(ifMap, weight, bias, C, M)
    return self.fullRead(M)

CNN.pool = pool
CNN.conv = conv
CNN.full = full

배열(list)이 아닌 변수는 ```register_map```을 통해 할당합니다. 배열은 ```struct```모듈을 이용해 배열 통째로 IEEE 754 형식에 맞추어 float을 byte로 변환한 후 해당하는 주소에 write합니다.

shape가 $\left(X_{1}, X_{2}, \cdots, X_{n} \right)$인 $n$차원 배열에 대하여
```multiDimArray```$\left[x_{1}, x_{2}, \cdots, x_{n} \right]$은 ```singleDimArray```$\left[ \left( \left( \left( x_{n}X_{n-1} + x_{n-1}\right) \times \cdots \right) \times X_{2} + x_{2} \right)\times X_{1}+x_{1} \right]$로 변환할 수 있고 나머지 정리에 의해 역도 성립합니다. 마찬가지로 ```struct```모듈을 이용해 해당하는 주소의 값을 read한 후 IEEE 754 형식에 맞추어 byte에서 float으로 변환합니다.

In [ ]:
def poolWrite(self, ifMap, C, H, U, R):
    self.pool_ip.register_map.C = C
    self.pool_ip.register_map.H = H
    self.pool_ip.register_map.U = U
    self.pool_ip.register_map.R = R

    ifMap = np.array(ifMap, dtype=np.float32).flatten()
    self.pool_ip.write(poolAddress["ifMap"], struct.pack(f"{ifMap.size}f", *ifMap))

def poolRead(self, C, E):
    ofMap = [[[0 for _ in range(E)] for _ in range(E)] for _ in range(C)]
    for u in range(C):
        for x in range(E):
            for y in range(E):
                readedFloat = self.pool_ip.read(poolAddress["ofMap"] + ((u * E + x) * E + y) * 4)
                readedByte = readedFloat.to_bytes(4, byteorder="little")
                ofMap[u][x][y] = struct.unpack("" readedByte)[0]
    return ofMap

CNN.poolWrite = poolWrite
CNN.poolRead = poolRead

Activation은 LeNet-5 논문에서 소개된대로 $f \left( a \right) = A \textrm{tanh} \left(Sa \right) $를 사용합니다. ($A \simeq 1.7159$, $S = \frac{2}{3} $)

In [ ]:
def convWrite(self, ifMap, _filter, bias, H, M, C, R, U):
    self.conv_ip.register_map.H = H
    self.conv_ip.register_map.M = M
    self.conv_ip.register_map.C = C
    self.conv_ip.register_map.R = R
    self.conv_ip.register_map.U = U

    ifMap = np.array(ifMap, dtype=np.float32).flatten()
    self.conv_ip.write(convAddress["ifMap"], struct.pack(f"{ifMap.size}f", *ifMap))

    _filter = np.array(_filter, dtype=np.float32).flatten()
    self.conv_ip.write(convAddress["_filter"], struct.pack(f"{_filter.size}f", *_filter))

    bias = np.array(bias, dtype=np.float32).flatten()
    self.conv_ip.write(convAddress["bias"], struct.pack(f"{bias.size}f", *bias))

def convRead(self, M, E):
    ofMap = [[[0 for _ in range(E)] for _ in range(E)] for _ in range(M)]
    for u in range(M):
        for x in range(E):
            for y in range(E):
                readedFloat = self.conv_ip.read(convAddress["ofMap"] + ((u * E + x) * E + y) * 4)
                readedByte = readedFloat.to_bytes(4, byteorder="little")
                ofMap[u][x][y] = 1.7159 * math.tanh(2 / 3 * struct.unpack('f', readedByte)[0])
    return ofMap

CNN.convWrite = convWrite
CNN.convRead = convRead

In [ ]:
def fullWrite(self, ifMap, weight, bias, C, M):
    self.full_ip.register_map.C = C
    self.full_ip.register_map.M = M

    ifMap = np.array(ifMap, dtype=np.float32).flatten()
    self.full_ip.write(fullAddress['ifMap'], struct.pack('%sf' % ifMap.size, *ifMap))

    weight = np.array(weight, dtype=np.float32).flatten()
    self.full_ip.write(fullAddress['weight'], struct.pack('%sf' % weight.size, *weight))

    bias = np.array(bias, dtype=np.float32).flatten()
    self.full_ip.write(fullAddress['bias'], struct.pack('%sf' % bias.size, *bias))

def fullRead(self, M):
    ofMap = [0 for _ in range(M)]
    for j in range(M):
        readedFloat = self.full_ip.read(fullAddress['ofMap'] + (j* 4))
        readedByte = readedFloat.to_bytes(4, byteorder="little")
        ofMap[j] = struct.unpack('f', readedByte)[0]
    return ofMap

CNN.fullWrite = fullWrite
CNN.fullRead = fullRead

In [ ]:
model = CNN()

# 값 확인
사용자로부터 0 이상의 정수 ```idx```를 입력받습니다.

keras에서는 shape가 $\left(32, 32, 1\right)$이었던 반면 여기서는 shape가 $\left(1, 32, 32\right)$이므로 전치(transpose) 과정을 거쳐야 합니다. 따라서 ```ifMap```과 convolution 과정에서 사용하는 weightList는 전치한 값을 사용합니다. 각 레이어의 output feature map은 ```C1```, ```S2``` 등에 저장됩니다.

In [ ]:
idx = int(input())
ifMap = test_images[idx].T.reshape(32, 32).tolist()

C1 = model.conv(ifMap, np.array(weightList[0]).T.tolist(), weightList[1], 1, 1, 32, 5, 6)
S2 = model.pool(C1, 2, 6, 28, 2)
C3 = model.conv(S2, np.array(weightList[2]).T.tolist(), weightList[3], 1, 6, 14, 5, 16)
S4 = model.pool(C3, 2, 16, 10, 2)
C5 = model.conv(S4, np.array(weightList[4]).T.tolist(), weightList[5], 1, 16, 5, 5, 120)
F6 = model.full(np.array(C5).reshape(120), weightList[6], weightList[7], 120, 84)
ofMap = model.full(F6, weightList[8], weightList[9], 84, 10)

print(f"예측: {np.argmax(ofMap)}, 정답: {test_labels[idx]}")
pixels = test_images[idx].reshape(32, 32)
plt.imshow(pixels, cmap='gray')
plt.show()